#### Required python libraries

In [ ]:
# pip install pandas numpy matplotlib seaborn

#### Defining the imports

In [9]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

#### Loading the provided datasets

* Features Dataset : features.csv :- Additional data related to the store, department, and regional activity for the given dates.
* Train Dataset : train.csv :- This is the historical training data, which covers 2010–02–05 to 2012-08-17.
* Test Dataset : test.csv :- Similar to train.csv, without the weekly sales column
* Stores Dataset : stores.csv :- Contains anonymized information about the 45 stores, indicating the type and size of the store.

In [10]:
df_features = pd.read_csv("dataset/features.csv")
df_store = pd.read_csv("dataset/stores.csv")
df_train = pd.read_csv("dataset/train.csv")
df_test = pd.read_csv("dataset/train.csv")

In [11]:
# Replacing Column Name spaces with "_"
df_features.columns = df_features.columns.str.replace(' ', '_')
df_store.columns = df_store.columns.str.replace(' ', '_')
df_train.columns = df_train.columns.str.replace(' ', '_')